# 测试脚本（安卓）

In [1]:
from impala.dbapi import connect
from impala.util import as_pandas
import pandas as pd
import numpy as np
import requests
import pymysql
import seaborn as sns

In [2]:
from create_plan_dt_mr_gdt import *

In [3]:
create_plan_mr_gdt_dt = GDTCreatePlanDt()

[INFO]: 近期优质计划数据提取完毕！


In [4]:
df = create_plan_mr_gdt_dt.Process()

基准配比： 5 基准素材量： 30
评分素材量： 34
大R素材量： 7


起量低付费素材量： 14


纯高量素材量 2


素材池结果 
     image_id    weight  image_run_date_amount
0      39345  2.660873               40436.05
1      39302  2.641224               42778.84
2      37282  2.832325               19992.78
3      38489  2.782924               25883.14
4      37812  2.022215              116586.81
5      37965  2.747950               30053.34
6      38335  2.756220               29067.28
7      39417  1.000000               17802.97
8      37774  3.803567               17566.38
9      38835  2.648860               41868.46
10     38396  3.435366               50493.36
11     38670  2.948177                6179.20
12     34335  2.593139               48512.31
13     39377  2.674939               38758.87
14     38613  2.734167               31696.81
15     39573  1.000000               25382.77
16     38141  2.740072               30992.63
17     39425  3.637953               32376.71
18     34373  3.639814               32210.25
19     36075  2.908798               97582.66
20     33114  2.703607    

# 测试逻辑（安卓）

## 获取大盘基准

In [38]:
def base_market(amount_thresh):
    """[逻辑：大盘决定 基准配比值 & 素材量]

    Returns:
        [float]: [基准配比值，素材量]
    """
    for day in range(1, 31):
        sum_amount, sum_roi, sum_cost = get_roi_my(0, day, day) 
        sum_roi = sum_roi * 100
        if sum_amount > amount_thresh*day or day >= 30:  ## 大盘达标 or 近30日大盘
            break
    base_prop, base_num = 5, 30
    print(sum_roi, sum_cost)
    # 规则1：大盘 决定 配比基准值 & 素材基准量
    if sum_cost != 0:
        if (sum_roi <= 1.5) & (sum_cost >= 10000):
            base_prop, base_num =3, 20 
        elif ((sum_roi <= 1.5) & (sum_cost < 10000)) or ((sum_roi <=1.7) & (sum_cost >= 8000)):
            base_prop, base_num =4, 25
        elif ((sum_roi <= 1.7) & (sum_cost < 8000)) or ((sum_roi <=2.0) & (sum_cost >= 6000)):
            base_prop, base_num =5, 30
        elif ((sum_roi <= 2.0) & (sum_cost < 6000)) or ((sum_roi <= 2.3) & (sum_cost >= 5000)):
            base_prop, base_num =6, 35
        elif ((sum_roi <= 2.3) & (sum_cost < 5000)) or ((sum_roi <= 2.5) & (sum_cost >= 4000)):
            base_prop, base_num =7, 40
        else:
            base_prop, base_num =8, 45

    # 规则2：个人盘 修正 极端roi的情况
    for day in range(1, 31):
        sum_amount, sum_roi, sum_cost = get_roi_my(0, day, day) 
        sum_roi = sum_roi * 100
        if sum_amount > amount_thresh*day or day >= 30:
            if sum_roi > 7.0 or sum_roi < 0.5:
                base_prop, base_num = 5, 30   ## 个人数据异常高、低，不稳定时，采取保守策略
            break

In [39]:
base_market(20000)

1.6519 3878.322581


(5, 30)

## 获取总素材量

In [115]:
def image_candidate():
    """[逻辑：素材分级 近期评分 & 近期递补]

    Returns:
        [list]: [备选素材]
    """
    # 大盘基准
    base_prop, base_num = base_market(20000)

    # 素材储备
    ## 评分素材
    image_pf = np.intersect1d(get_score_image(), get_good_image())  ## 评分计划需满足：1）历史有过600评分，且当前评分580以上 and 2）有付费
    print("评分素材量：", len(image_pf))
    pf = pd.DataFrame(image_pf, columns=['image_id'])
    ## 近期递补 - 大R
    image_dr = get_R_image()  ## 大R计划需满足：1）消耗4000以上 and 2）平均付费金额1000以上 or 3）存在单用户付费金额3000以上
    print("大R素材量：", len(image_dr))
    dr = pd.DataFrame(image_dr, columns=['image_id'])
    ## 近期递补 - 中量低付费
    image_lz = get_lz_image()  ## 有量低付费素材需满足：1）素材总消耗10000以上 and 2）素材总回款1个点以下 and 3）素材有18的付费
    print("起量低付费素材量：", len(image_lz))
    lz = pd.DataFrame(image_lz, columns=['image_id'])
    ## 近期递补 - 纯高量
    image_lg = get_lg_image()  ## 纯高量素材需满足：1）素材总消耗100000以上
    print("纯高量素材量", len(image_lg))
    lg = pd.DataFrame(image_lg, columns=['image_id'])
    print("总素材量：", len(list(set(image_pf.tolist() + image_dr.tolist() + image_lz.tolist() + image_lg.tolist()))), "\n", list(set(image_pf.tolist() + image_dr.tolist() + image_lz.tolist() + image_lg.tolist())))
    # 素材加权
    dr['weight'], pf['weight'], lg['weight'], lz['weight'] = 4, 3, 2, 1  ## 初始权重（可调）
    df = dr.append(pf).append(lg).append(lz)
    df.reset_index(drop=True, inplace=True)
    df['weight'] = df.groupby('image_id')['weight'].transform('max')  ## TODO:素材分级加权
    df['image_id'] = df['image_id'].astype(int)
    df.drop_duplicates(inplace=True)
    df.reset_index(drop=True, inplace=True)
    # print("分级加权结果", len(df), "\n", df)

    df = pd.merge(df, get_baseimage_amount(df['image_id'].values), on=['image_id'], how='left')  ## 2选1：平衡大盘消耗
    # df['image_run_date_amount'] = df['image_id'].apply(get_amount_perimage_my)  ## 2选1：平衡个人盘消耗
    df['weight'] = df.apply(lambda x:x.weight * (1 - x.image_run_date_amount / df['image_run_date_amount'].max()) if x.weight > 1 else x.weight, axis=1)  ## TODO:素材平衡消耗的降权
    df = df.sample(n=base_num, weights=df['weight']).reset_index(drop=True)  ## 加权采样
    print("素材池结果", "\n", df)

In [116]:
image_candidate()

1.6587999999999998 3982.453125


评分素材量： 35
大R素材量： 12


起量低付费素材量： 14


纯高量素材量 2
总素材量： 51 
 [39425.0, 39427, 39302.0, 37774.0, 38670.0, 39183.0, 39441, 39573, 38550.0, 38554, 38683.0, 34335.0, 36767.0, 37282.0, 37155.0, 38652.0, 37926.0, 38823.0, 33066, 39345.0, 35378, 38835.0, 37812.0, 37046.0, 38838, 39356, 38335.0, 34373.0, 37965.0, 38991.0, 39377.0, 38354.0, 38100.0, 38613.0, 38868, 39512.0, 38489.0, 33114.0, 38622, 38624.0, 39138.0, 39141, 36075.0, 39022, 39279, 39153.0, 39282, 39417, 38396.0, 38141.0, 39423]
素材池结果 
     image_id    weight  image_run_date_amount
0      39345  3.547830               40436.05
1      38613  2.734167               31696.81
2      38683  2.928479                8527.87
3      34373  3.618886               34081.74
4      39138  2.964713                4207.45
5      38622  3.331220               59806.85
6      37155  2.661147               40403.41
7      39573  1.000000               27932.88
8      37812  2.017058              117201.68
9      39153  2.986364                1625.95
10     37282  2.830078               2

## 获取总ROI

In [5]:
def get_roi(x, y, z):
    """ 数据：大盘roi与大盘pay_cost两个指标 """
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
       SELECT
            aa.launch_op_id AS 'launch_op_id',
            aa.media_id AS 'media_id',
            sum( aa.amount ) AS 'amount',
            IFNULL( sum( aa.create_role_num ), 0 ) AS 'create_role_num',
            IFNULL( sum( bb.pay_role_user_num ), 0 ) AS 'pay_num',
            IFNULL( sum( bb.new_role_money ), 0 ) AS 'pay_sum',
            (
         CASE
                    WHEN ifnull( sum( aa.pay_role_user_num ), 0 )= 0 THEN
                            IFNULL( sum( aa.amount ), 0 ) ELSE IFNULL( sum( aa.amount ), 0 ) / ifnull( sum( aa.pay_role_user_num ), 0 )
                                                    END 
                                                    ) AS 'create_role_pay_cost', 
            (
         CASE
                    WHEN ifnull( sum( aa.amount ), 0 )= 0 THEN
                            0 ELSE IFNULL( sum( aa.new_role_money ), 0 ) / ifnull( sum( aa.amount ), 0 )
                                                    END 
                                                    ) AS 'create_role_roi' 
        FROM
            (
            SELECT
                a.game_id,
                a.channel_id,
                a.source_id,
                b.launch_op_id,
                a.media_id,
                IFNULL( sum( a.amount ), 0 ) AS amount,
                IFNULL( sum( create_role_num ), 0 ) AS create_role_num,
                IFNULL( sum( a.new_role_money ), 0 ) AS new_role_money,
                IFNULL( sum( a.pay_role_user_num ), 0 ) AS pay_role_user_num

            FROM
                db_stdata.st_lauch_report a
                INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                AND a.source_id = b.source_id 
                AND a.channel_id = b.chl_user_id
            WHERE
                a.tdate_type = 'day' 
                AND a.tdate <= DATE_SUB( date( NOW()), INTERVAL {} DAY )  AND a.tdate >= DATE_SUB( date( NOW()), INTERVAL {} DAY ) 
                AND a.media_id = 16
                AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                AND b.launch_op_id=13268  ## 可选：是否只看个人数据
            GROUP BY
                a.game_id,
                a.channel_id,
                a.source_id
            ) aa
            LEFT JOIN (
                SELECT
                    c.game_id,
                    c.channel_id,
                    c.source_id,
                    sum( c.create_role_money ) new_role_money,
                    IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                FROM
                    db_stdata.st_game_days c 
                WHERE
                    c.report_days = {} 
                    AND c.tdate = date( NOW() - INTERVAL 24 HOUR ) 
                    AND c.tdate_type = 'day' 
                    AND c.query_type = 13 
                GROUP BY
                    c.game_id,
                    c.channel_id,
                    c.source_id 
                HAVING
                ( new_role_money > 0 OR pay_role_user_num > 0 )
            ) bb ON aa.game_id = bb.game_id 
            AND aa.channel_id = bb.channel_id 
            AND aa.source_id = bb.source_id 
        GROUP BY
            aa.launch_op_id,
            aa.media_id
    '''
    finalSql = sql.format(x, y, z)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()

    return result_df['amount'].values[0] / z, result_df['create_role_roi'].values[0], result_df['create_role_pay_cost'].values[0]

In [6]:
get_roi(0, 1, 1)  ## 输出：平均每天消耗、总回款、总付费成本

(119077.0, 0.016006, 4106.103448)

## 获取单素材ROI

In [7]:
def get_roi_my_image(x, y, z):
    """ 数据：个人盘roi指标 """

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
       SELECT
            aa.launch_op_id AS 'launch_op_id',
            aa.media_id AS 'media_id',
            sum( aa.amount ) AS 'amount',
            IFNULL( sum( aa.create_role_num ), 0 ) AS 'create_role_num',
            IFNULL( sum( bb.pay_role_user_num ), 0 ) AS 'pay_num',
            IFNULL( sum( bb.new_role_money ), 0 ) AS 'pay_sum',
            (
         CASE
                    WHEN ifnull( sum( aa.pay_role_user_num ), 0 )= 0 THEN
                            IFNULL( sum( aa.amount ), 0 ) ELSE IFNULL( sum( aa.amount ), 0 ) / ifnull( sum( aa.pay_role_user_num ), 0 )
                                                    END 
                                                    ) AS 'create_role_pay_cost', 
            (
         CASE
                    WHEN ifnull( sum( aa.amount ), 0 )= 0 THEN
                            0 ELSE IFNULL( sum( aa.new_role_money ), 0 ) / ifnull( sum( aa.amount ), 0 )
                                                    END 
                                                    ) AS 'create_role_roi' 
        FROM
            (
            SELECT
                a.game_id,
                a.channel_id,
                a.source_id,
                b.launch_op_id,
                a.media_id,
                IFNULL( sum( a.amount ), 0 ) AS amount,
                IFNULL( sum( create_role_num ), 0 ) AS create_role_num,
                IFNULL( sum( a.new_role_money ), 0 ) AS new_role_money,
                IFNULL( sum( a.pay_role_user_num ), 0 ) AS pay_role_user_num

            FROM
                db_stdata.st_lauch_report a
                INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                AND a.source_id = b.source_id 
                AND a.channel_id = b.chl_user_id
            WHERE
                a.tdate_type = 'day' 
                AND a.tdate <= DATE_SUB( date( NOW()), INTERVAL {} DAY )  AND a.tdate >= DATE_SUB( date( NOW()), INTERVAL {} DAY ) 
                AND a.media_id = 16
                AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                AND b.image_id = 38622
                AND b.launch_op_id=13268  ## 只看个人盘数据
            GROUP BY
                a.game_id,
                a.channel_id,
                a.source_id
            ) aa
            LEFT JOIN (
                SELECT
                    c.game_id,
                    c.channel_id,
                    c.source_id,
                    sum( c.create_role_money ) new_role_money,
                    IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                FROM
                    db_stdata.st_game_days c 
                WHERE
                    c.report_days = {} 
                    AND c.tdate = date( NOW() - INTERVAL 24 HOUR ) 
                    AND c.tdate_type = 'day' 
                    AND c.query_type = 13 
                GROUP BY
                    c.game_id,
                    c.channel_id,
                    c.source_id 
                HAVING
                ( new_role_money > 0 OR pay_role_user_num > 0 )
            ) bb ON aa.game_id = bb.game_id 
            AND aa.channel_id = bb.channel_id 
            AND aa.source_id = bb.source_id 
        GROUP BY
            aa.launch_op_id,
            aa.media_id
    '''
    finalSql = sql.format(x, y, z)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()

    return result_df['amount'].values[0], result_df['create_role_roi'].values[0]

In [8]:
get_roi_my_image(0, 3, 3)  ## 输出：平均每天消耗、总回款、总付费成本

(14298.08, 0.005036)

## 获取单素材AMOUNT

In [9]:
def get_amount_perimage_my(image_id):
    """ 数据：个人盘指定素材的消耗量 """

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
       SELECT
            aa.launch_op_id AS 'launch_op_id',
            aa.media_id AS 'media_id',
            sum( aa.amount ) AS 'amount',
            IFNULL( sum( aa.create_role_num ), 0 ) AS 'create_role_num',
            IFNULL( sum( bb.pay_role_user_num ), 0 ) AS 'pay_num',
            IFNULL( sum( bb.new_role_money ), 0 ) AS 'pay_sum',
            (
         CASE
                    WHEN ifnull( sum( aa.pay_role_user_num ), 0 )= 0 THEN
                            IFNULL( sum( aa.amount ), 0 ) ELSE IFNULL( sum( aa.amount ), 0 ) / ifnull( sum( aa.pay_role_user_num ), 0 )
                                                    END 
                                                    ) AS 'create_role_pay_cost', 
            (
         CASE
                    WHEN ifnull( sum( aa.amount ), 0 )= 0 THEN
                            0 ELSE IFNULL( sum( aa.new_role_money ), 0 ) / ifnull( sum( aa.amount ), 0 )
                                                    END 
                                                    ) AS 'create_role_roi' 
        FROM
            (
            SELECT
                a.game_id,
                a.channel_id,
                a.source_id,
                b.launch_op_id,
                a.media_id,
                IFNULL( sum( a.amount ), 0 ) AS amount,
                IFNULL( sum( create_role_num ), 0 ) AS create_role_num,
                IFNULL( sum( a.new_role_money ), 0 ) AS new_role_money,
                IFNULL( sum( a.pay_role_user_num ), 0 ) AS pay_role_user_num

            FROM
                db_stdata.st_lauch_report a
                INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                AND a.source_id = b.source_id 
                AND a.channel_id = b.chl_user_id
            WHERE
                a.tdate_type = 'day' 
                AND a.tdate <= DATE_SUB( date( NOW()), INTERVAL 0 DAY )  AND a.tdate >= DATE_SUB( date( NOW()), INTERVAL 30 DAY ) 
                AND a.media_id = 16
                AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL )
                AND b.image_id = {}
                # AND b.launch_op_id=13268  ## 只看个人盘数据
            GROUP BY
                a.game_id,
                a.channel_id,
                a.source_id
            ) aa
            LEFT JOIN (
                SELECT
                    c.game_id,
                    c.channel_id,
                    c.source_id,
                    sum( c.create_role_money ) new_role_money,
                    IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                FROM
                    db_stdata.st_game_days c 
                WHERE
                    c.report_days = 30 
                    AND c.tdate = date( NOW() - INTERVAL 24 HOUR ) 
                    AND c.tdate_type = 'day' 
                    AND c.query_type = 13 
                GROUP BY
                    c.game_id,
                    c.channel_id,
                    c.source_id 
                HAVING
                ( new_role_money > 0 OR pay_role_user_num > 0 )
            ) bb ON aa.game_id = bb.game_id 
            AND aa.channel_id = bb.channel_id 
            AND aa.source_id = bb.source_id 
        GROUP BY
            aa.launch_op_id,
            aa.media_id
    '''
    finalSql = sql.format(int(image_id))
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()

    return result_df['amount'].values[0]

In [10]:
get_amount_perimage_my(39022.0)

2342.9

## 获取dws表结构

In [11]:
conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
cursor = conn.cursor()
sql_engine = 'set hive.execution.engine=tez'
sql = 'select * from dws.dws_image_score_d where media_id=16 and score>=500 and dt>=date_sub(current_date, 1)'
cursor.execute(sql_engine)
cursor.execute(sql)
result = as_pandas(cursor)
result.columns

Index(['dws_image_score_d.image_id', 'dws_image_score_d.image_name',
       'dws_image_score_d.image_run_date_amount',
       'dws_image_score_d.image_create_role_pay_num',
       'dws_image_score_d.image_create_role_num',
       'dws_image_score_d.image_create_role_pay_sum',
       'dws_image_score_d.image_source_num',
       'dws_image_score_d.image_create_role_pay_rate',
       'dws_image_score_d.image_create_role_cost',
       'dws_image_score_d.image_create_role_pay_cost',
       'dws_image_score_d.image_valid_source_num',
       'dws_image_score_d.image_valid_source_rate',
       'dws_image_score_d.image_pay_sum_ability',
       'dws_image_score_d.image_pay_num_ability',
       'dws_image_score_d.image_create_role_roi',
       'dws_image_score_d.image_create_role_retain_1d',
       'dws_image_score_d.model_run_datetime', 'dws_image_score_d.data_win',
       'dws_image_score_d.score', 'dws_image_score_d.image_launch_time',
       'dws_image_score_d.image_source_total_num',
       

## 获取base消耗

In [12]:
def get_baseimage_amount(base_image):
    """ 数据：提取指定素材的素材累计消耗 """
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = 'select image_id,image_run_date_amount,dt from dws.dws_image_score_d where media_id=16 and dt>=date_sub(current_date, 30)'
    cursor.execute(sql_engine)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result = result[result['image_id'].isin(base_image)]
    result = result.sort_values('dt')
    result = result.drop_duplicates(['image_id'], keep='last')  ## TODO:保留最近的1次
    # 关闭链接
    cursor.close()
    conn.close()

    return result[['image_id','image_run_date_amount']]

In [13]:
get_baseimage_amount([34373,36075,37774,38396])

,image_id,image_run_date_amount
51121,36075,95351.36
51115,34373,30409.71
51416,37774,17566.38
52012,38396,60015.40


## 4部分素材提取测试
* 评分素材
* 大R素材
* 低充素材

In [14]:
import datetime

### 评分素材

In [104]:
def get_score_image():
    """ 数据：近15天的历史有过600评分且当前评分580以上 """

    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = 'select image_id,dt from dws.dws_image_score_d where media_id=16 and score>=600 and dt>=date_sub(current_date, 30)'
    cursor.execute(sql_engine)
    cursor.execute(sql)
    result_1 = as_pandas(cursor)
    sql = 'select image_id,dt from dws.dws_image_score_d where media_id=16 and score>=550 and dt=current_date'
    cursor.execute(sql_engine)
    cursor.execute(sql)
    result_2 = as_pandas(cursor)
    result = np.intersect1d(result_1['image_id'], result_2['image_id'])
    # 关闭链接
    cursor.close()
    conn.close()

    return result

def get_good_image():
    """ 数据：近15天的付费计划素材 """

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            b.channel_id,
            b.source_id,
            b.tdate,
            b.amount 
        FROM
            db_stdata.st_lauch_report b
        WHERE
            b.tdate >= date( NOW() - INTERVAL 720 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 16 
            AND b.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
            AND b.amount >= 500
            AND b.pay_role_user_num >= 1
            AND b.new_role_money >= 6
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)

    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id
        FROM
            db_data_ptom.ptom_plan a
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
            AND a.media_id = 16 
            AND a.create_time >= date( NOW() - INTERVAL 720 HOUR )
        GROUP BY
            a.chl_user_id,
            a.source_id,
            a.image_id
    '''
    cur.execute(sql)
    result = pd.read_sql(sql, conn)
    result = pd.merge(result, result_df, on=['channel_id', 'source_id'], how='right')
    result['tdate'] = pd.to_datetime(result['tdate'])
    result = result.sort_values('tdate')
    result = result.drop_duplicates(['channel_id', 'source_id'], keep='last')
    # 关闭链接
    cur.close()
    conn.close()

    return np.unique(result['image_id'].values)

In [105]:
image_yz = np.intersect1d(get_score_image(), get_good_image())
print("评分素材", len(image_yz), '\n', image_yz.tolist())

评分素材 35 
 [33114.0, 34335.0, 34373.0, 36075.0, 36767.0, 37046.0, 37155.0, 37282.0, 37774.0, 37812.0, 37926.0, 37965.0, 38100.0, 38141.0, 38335.0, 38354.0, 38396.0, 38489.0, 38550.0, 38613.0, 38624.0, 38652.0, 38670.0, 38683.0, 38823.0, 38835.0, 38991.0, 39138.0, 39153.0, 39183.0, 39302.0, 39345.0, 39377.0, 39425.0, 39512.0]


### 大R素材

In [106]:
def get_R_image():
    """ 数据：近30天的已关停大R计划素材 """

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id AS source_id,
            a.image_id,
            b.tdate
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN db_stdata.st_lauch_report b ON a.chl_user_id = b.channel_id 
            AND a.source_id = b.source_id 
        WHERE
            a.create_time >= date( NOW() - INTERVAL 720 HOUR )
            AND b.tdate >= date( NOW() - INTERVAL 720 HOUR )
            AND b.tdate_type = 'day' 
            AND b.media_id = 16
            AND b.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL )
            AND b.amount >= 4000
            AND ((b.new_role_money / b.pay_role_user_num >= 2000) or (b.new_role_money >= 1000 and b.pay_role_user_num = 1))
    '''
    cur.execute(sql)
    result = pd.read_sql(sql, conn)
    result['tdate'] = pd.to_datetime(result['tdate'])
    result = result.sort_values('tdate')
    result = result.drop_duplicates(['channel_id', 'source_id'], keep='last')  ## TODO:保留最近的1次
    result = result[result['tdate'].dt.date < datetime.date.today()]
    
    # 关闭链接
    cur.close()
    conn.close()

    return np.unique(result['image_id'].values)

In [107]:
image_dr = get_R_image()
print("大R素材", len(image_dr), '\n', image_dr.tolist())

大R素材 12 
 [33066, 34373, 35378, 36075, 37046, 37774, 38396, 38554, 38622, 38838, 39345, 39425]


In [108]:
np.intersect1d(image_yz, image_dr)

array([34373., 36075., 37046., 37774., 38396., 39345., 39425.])

### 量中低充素材

In [109]:
def get_lz_image():
    """ 数据：近30天的有量低充值计划素材 """
    
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN
            db_data_ptom.ptom_image_info b
        on a.image_id = b.image_id
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
            AND a.media_id = 16
            AND b.create_time >= date( NOW() - INTERVAL 720 HOUR )  ## 素材：创建时间在30天以内
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    # 关闭链接
    cur.close()
    conn.close()

    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = 'select image_id,image_run_date_amount,image_create_role_pay_sum,image_create_role_roi,dt from dws.dws_image_score_d where media_id=16 and dt>=date_sub(current_date, 30)'
    cursor.execute(sql_engine)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result = result.sort_values('dt')
    result = result.drop_duplicates(['image_id'], keep='last')  ## TODO:保留最近的1次
    result = result[(result['image_run_date_amount']>10000) & (result['image_create_role_pay_sum']>18) & (result['image_create_role_roi']<0.01)]
    result = np.intersect1d(result['image_id'], result_df['image_id'])
    # 关闭链接
    cur.close()
    conn.close()

    return result

In [110]:
image_lz = get_lz_image()
print("量中低充素材", len(image_lz), '\n', image_lz.tolist())

量中低充素材 14 
 [38622, 38835, 38868, 39022, 39141, 39183, 39279, 39282, 39356, 39417, 39423, 39427, 39441, 39573]


In [111]:
np.intersect1d(image_yz, image_lz)

array([38835., 39183.])

### 高消耗素材

In [112]:
def get_ld_image():
    """ 数据：近30天的高量计划素材 """
    
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN
            db_data_ptom.ptom_image_info b
        on a.image_id = b.image_id
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
            AND a.media_id = 16
            AND b.create_time >= date( NOW() - INTERVAL 720 HOUR )  ## 素材：创建时间在30天以内
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    # 关闭链接
    cur.close()
    conn.close()

    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = 'select image_id,image_run_date_amount,dt from dws.dws_image_score_d where media_id=16 and dt>=date_sub(current_date, 30)'
    cursor.execute(sql_engine)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result = result.sort_values('dt')
    result = result.drop_duplicates(['image_id'], keep='last')  ## TODO:保留最近的1次
    result = result[result['image_run_date_amount']>80000]
    result = np.intersect1d(result['image_id'], result_df['image_id'])
    # 关闭链接
    cur.close()
    conn.close()

    return result

In [113]:
image_ld = get_ld_image()
print("量大素材", len(image_ld), '\n', image_ld.tolist())

量大素材 3 
 [38823, 38868, 38991]


In [114]:
np.intersect1d(image_yz, image_lz)

array([38835., 39183.])

## 大中小R标准区分

In [68]:
# def get_user_info(cursor, start_date, nday):
#     """
#     :param cursor: hive数仓的游标
#     :param start_date: 指定的统计开始日期
#     :param nday: 指定的开区时间周期，如3代表统计开区3天内的角色行为数据
#     :return: user_info
#     """
#     sql_engine = 'set hive.execution.engine=tez'
#     originSql = '''
#     SELECT
#         A.user_id AS user_id,
#         A.mgame_id AS mgame_id,
#         A.cp_server_no AS cp_server_no,
#         A.role_id AS role_id,
#         A.create_role_time AS create_role_time,
#         A.channel_id AS channel_id,
#         A.source_id AS source_id,
#         A.money AS money,
#         nvl(B.online_time,0) AS online_time,
#         nvl(B.action,0) AS action 
#     FROM
#         (
#         SELECT
#             user_id,
#             mgame_id,
#             cp_server_no,
#             role_id,
#             create_role_time,
#             channel_id,
#             source_id,
#             SUM(nvl ( c.money, 0 )) AS money 
#         FROM
#             (
#             SELECT
#                 a.mgame_id,
#                 a.cp_server_no,
#                 a.cp_role_id,
#                 a.create_role_time,
#                 b.channel_id,
#                 b.source_id,
#                 b.user_id,
#                 b.role_id AS role_id 
#             FROM
#                 dim.dim_mwcprole_info a,
#                 (
#                 SELECT
#                     a.cp_role_id,
#                     a.cp_server_no,
#                     a.role_id,
#                     a.user_id,
#                     a.channel_id,
#                     a.source_id,
#                     b.mgame_id 
#                 FROM
#                     dim.dim_mwrole_info a
#                     JOIN dim.dim_mwgame_info b ON a.game_id = b.game_id 
#                 WHERE
#                     a.dt >= '{start_date}' 
#                 ) b 
#             WHERE
#                 a.dt >= '{start_date}'
#                 AND a.mgame_id = 1056 
#                 AND a.cp_role_id = b.cp_role_id 
#                 AND a.cp_server_no = b.cp_server_no 
#                 AND a.mgame_id = b.mgame_id 
#                 AND a.create_role_time >= '{start_date}' 
#             ) a
#             LEFT JOIN (
#             SELECT
#                 a.role_id,
#                 a.money,
#                 a.pay_time 
#             FROM
#                 ods.ods_product_order AS a
#                 LEFT JOIN dim.dim_mwgame_info AS b ON a.game_id = b.game_id 
#             WHERE
#                 a.dt >= '{start_date}'
#                 AND b.mgame_id = 1056 
#            UNION ALL
#             SELECT
#                 a.role_id,
#                 a.money,
#                 a.pay_time
#             FROM
#                 ods.ods_ios_product_order AS a
#                 LEFT JOIN dim.dim_mwgame_info AS b ON a.game_id = b.game_id 
#             WHERE
#                 a.dt >= '{start_date}'
#                 AND b.mgame_id = 1056   
#             ) c ON a.role_id = c.role_id 
#         WHERE
#             c.pay_time IS NULL 
#             OR (
#                 unix_timestamp( c.pay_time ) - unix_timestamp( a.create_role_time ) >= 0 
#                 AND unix_timestamp( c.pay_time ) - unix_timestamp( a.create_role_time ) <= {nday} * 86400
#                 AND a.create_role_time IS NOT NULL 
#                 AND unix_timestamp() - unix_timestamp( a.create_role_time ) >= {nday} * 86400
#             ) 
#         GROUP BY
#             a.user_id,
#             a.mgame_id,
#             a.cp_server_no,
#             a.role_id,
#             a.create_role_time,
#             a.channel_id,
#             a.source_id
#         ) A
#         LEFT JOIN (
#         SELECT
#             role_id,
#         IF
#             ( sum( online_time ) >= 0, sum( online_time ), 0 ) AS online_time,
#             COUNT(
#             DISTINCT date_format( login_time, 'yyyy-MM-dd' )) AS action 
#         FROM
#             (
#             SELECT
#                 a.role_id,
#                 c.login_time,
#                 (
#                 CASE

#                         WHEN unix_timestamp(
#                             MAX( c.active_time )) - unix_timestamp( c.login_time ) > 86400 THEN
#                             86400 ELSE unix_timestamp(
#                             MAX( c.active_time )) - unix_timestamp( c.login_time ) 
#                         END 
#                         ) AS online_time 
#                     FROM
#                         (
#                         SELECT
#                             a.mgame_id,
#                             a.cp_server_no,
#                             a.cp_role_id,
#                             a.create_role_time,
#                             b.channel_id,
#                             b.source_id,
#                             b.user_id,
#                             b.role_id AS role_id 
#                         FROM
#                             dim.dim_mwcprole_info a,
#                             (
#                             SELECT
#                                 a.cp_role_id,
#                                 a.cp_server_no,
#                                 a.role_id,
#                                 a.user_id,
#                                 a.channel_id,
#                                 a.source_id,
#                                 b.mgame_id 
#                             FROM
#                                 dim.dim_mwrole_info a
#                                 JOIN dim.dim_mwgame_info b ON a.game_id = b.game_id 
#                             WHERE
#                                 a.dt >= '{start_date}' 
#                             ) b 
#                         WHERE
#                             a.dt >= '{start_date}'
#                             AND a.mgame_id = 1056                     
#                             AND a.cp_role_id = b.cp_role_id 
#                             AND a.cp_server_no = b.cp_server_no 
#                             AND a.mgame_id = b.mgame_id 
#                             AND a.create_role_time >= '{start_date}' 
#                         ) a
#                         LEFT JOIN (
#                         SELECT
#                             role_id,
#                             active_time,
#                             login_time 
#                         FROM
#                             ods.ods_heart_beat a
#                             LEFT JOIN dim.dim_mwgame_info AS b ON a.game_id = b.game_id 
#                         WHERE
#                             a.dt >= '{start_date}'
#                             AND b.mgame_id = 1056 
#                         ) c ON a.role_id = c.role_id 
#                     WHERE
#                         unix_timestamp( c.active_time ) - unix_timestamp( a.create_role_time ) >= 0 
#                         AND unix_timestamp( c.active_time ) - unix_timestamp( a.create_role_time ) <= {nday} * 86400
#                         AND a.create_role_time IS NOT NULL 
#                         AND unix_timestamp() - unix_timestamp( a.create_role_time ) >= {nday} * 86400
#                     GROUP BY
#                         a.role_id,
#                         c.login_time 
#                     ) a 
#                 GROUP BY
#                 role_id 
#         ) B ON A.role_id = B.role_id
#     '''
#     finalSql = originSql.format(start_date=start_date, nday=nday)
#     cursor.execute(sql_engine)
#     cursor.execute(finalSql)
#     result = as_pandas(cursor)
#     result['n_day'] = nday
#     result['action'] = result['action'].clip(0, nday)
#     # 计算大中小R分类标准
#     result.sort_values('money', ascending=False, inplace=True)
#     result['cum_sum'] = result['money'].cumsum()
#     big_r_money = result[result['cum_sum'] > result['money'].sum() * 0.7]['money'].max()
#     mid_r_money = result[result['cum_sum'] > result['money'].sum() * 0.9]['money'].max()
#     # 进行角色分类
#     result['user_type'] = result.apply(
#         lambda x: 'big_R' if x.money >= big_r_money else ('mid_R' if x.money >= mid_r_money else ('small_R' if
#                                                                                                   x.money > 0 else (
#             'rubbish' if ((x.n_day > 3) & (x.action <= 1)) | (x.online_time <= 600) else (
#                 'high_action' if (x.action >= 20) &
#                                  (x.online_time >= x.n_day * 1800) else 'mid_action')))), axis=1)
#     result.drop('cum_sum', axis=1, inplace=True)
#     result['big_r_money_standard'] = big_r_money
#     result['mid_r_money_standard'] = mid_r_money
#     return result

In [69]:
# conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
#                         password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
# cursor = conn.cursor()
# get_user_info(cursor, '2021-10-01', 1)

## 指定评分提取测试

In [26]:
def get_baseimage_score(base_image):
    """ 数据：提取指定素材的当前评分 """
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql = 'select image_id,score,dt from dws.dws_image_score_d where media_id=16 and dt>=date_sub(current_date, 30)'
    cursor.execute(sql_engine)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result = result[result['image_id'].isin(base_image)]
    result = result.sort_values('dt')
    result = result.drop_duplicates(['image_id'], keep='last')  ## TODO:保留最近的1次
    # 关闭链接
    cursor.close()
    conn.close()

    return result

In [27]:
base_image = [39302,38550,38835,39138,37774,39153,39441,38489,37812,39345,37282,36767] 

In [28]:
get_baseimage_score(base_image)

,image_id,score,dt
51228,39302,519,2021-10-14
51400,36767,550,2021-10-14
51410,37282,608,2021-10-14
51416,37774,675,2021-10-14
51419,37812,612,2021-10-14
51450,38489,554,2021-10-14
51453,38550,640,2021-10-14
51468,38835,550,2021-10-14
51482,39138,569,2021-10-14
51486,39153,620,2021-10-14


## 指定账号的roi测试

In [29]:
def get_roi_peraccount_my(ad_account_id):
    """ 数据：大盘roi与大盘pay_cost两个指标 """
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
       SELECT
            aa.launch_op_id AS 'launch_op_id',
            aa.media_id AS 'media_id',
            sum( aa.amount ) AS 'amount',
            IFNULL( sum( aa.create_role_num ), 0 ) AS 'create_role_num',
            IFNULL( sum( bb.pay_role_user_num ), 0 ) AS 'pay_num',
            IFNULL( sum( bb.new_role_money ), 0 ) AS 'pay_sum',
            (
         CASE
                    WHEN ifnull( sum( aa.pay_role_user_num ), 0 )= 0 THEN
                            IFNULL( sum( aa.amount ), 0 ) ELSE IFNULL( sum( aa.amount ), 0 ) / ifnull( sum( aa.pay_role_user_num ), 0 )
                                                    END 
                                                    ) AS 'create_role_pay_cost', 
            (
         CASE
                    WHEN ifnull( sum( aa.amount ), 0 )= 0 THEN
                            0 ELSE IFNULL( sum( aa.new_role_money ), 0 ) / ifnull( sum( aa.amount ), 0 )
                                                    END 
                                                    ) AS 'create_role_roi' 
        FROM
            (
            SELECT
                a.game_id,
                a.channel_id,
                a.source_id,
                b.launch_op_id,
                a.media_id,
                IFNULL( sum( a.amount ), 0 ) AS amount,
                IFNULL( sum( create_role_num ), 0 ) AS create_role_num,
                IFNULL( sum( a.new_role_money ), 0 ) AS new_role_money,
                IFNULL( sum( a.pay_role_user_num ), 0 ) AS pay_role_user_num

            FROM
                db_stdata.st_lauch_report a
                INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                AND a.source_id = b.source_id 
                AND a.channel_id = b.chl_user_id
            WHERE
                a.tdate_type = 'day' 
                AND a.tdate <= DATE_SUB( date( NOW()), INTERVAL 0 DAY )  AND a.tdate >= DATE_SUB( date( NOW()), INTERVAL 30 DAY ) 
                AND a.media_id = 16
                AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                AND b.ad_account_id = {}
                # AND b.launch_op_id=13268  ## 可选：是否只看个人数据
            GROUP BY
                a.game_id,
                a.channel_id,
                a.source_id
            ) aa
            LEFT JOIN (
                SELECT
                    c.game_id,
                    c.channel_id,
                    c.source_id,
                    sum( c.create_role_money ) new_role_money,
                    IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                FROM
                    db_stdata.st_game_days c 
                WHERE
                    c.report_days = 30
                    AND c.tdate = date( NOW() - INTERVAL 24 HOUR ) 
                    AND c.tdate_type = 'day' 
                    AND c.query_type = 13 
                GROUP BY
                    c.game_id,
                    c.channel_id,
                    c.source_id 
                HAVING
                ( new_role_money > 0 OR pay_role_user_num > 0 )
            ) bb ON aa.game_id = bb.game_id 
            AND aa.channel_id = bb.channel_id 
            AND aa.source_id = bb.source_id 
        GROUP BY
            aa.launch_op_id,
            aa.media_id
    '''
    finalSql = sql.format(ad_account_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()

    return result_df['amount'].values[0], result_df['pay_sum'].values[0], result_df['create_role_roi'].values[0]

In [30]:
get_roi_peraccount_my(7983)

(0.0, 304.0, 0.0)

In [31]:
get_roi_peraccount_my(7984)

(283082.92, 96668.0, 0.010986)

# 测试脚本（IOS）

In [1]:
from impala.dbapi import connect
from impala.util import as_pandas
import pandas as pd
import numpy as np
import requests
import pymysql
import seaborn as sns

In [2]:
from create_plan_dt_mr_gdt_ios import *

In [3]:
create_plan_mr_gdt_ios_dt = GDTCreatePlanDtIOS()

[INFO]: 近期优质计划数据提取完毕！


In [4]:
df = create_plan_mr_gdt_ios_dt.Process()

基准配比： 3 基准素材量： 10
评分素材量： 13
大R素材量： 7


起量低付费素材量： 13


纯高量素材量 2
总素材量： 30 
 [39425, 39561, 37774, 39183, 39442, 39577, 38554, 38683.0, 38396.0, 34335.0, 37926.0, 38823.0, 33066.0, 39345, 38835, 37812.0, 38838, 39606, 39612, 39614, 34373, 38991.0, 39377.0, 38489.0, 33114.0, 39138.0, 39654, 36075.0, 39417, 38652]


素材池结果 
    image_id    weight  image_run_date_amount
0     38835  1.747510               61357.66
1     39377  7.223624               47166.81
2     39417  1.919185               19638.89
3     37926  7.214591               47715.60
4     33114  7.472371               32054.78
5     39138  7.965993                2066.04
6     37774  5.821797               14435.04
7     38683  7.929138                4305.04
8     38489  7.533662               28331.24
9     34335  6.608474               84538.75
个性化配比结果： 
    image_id  score  prop
0     39138    615     5
1     38835    556     3
2     38683    593     4
3     38489    582     4
4     37926    583     4
5     37774    564     3
6     34335    602     5
7     33114    644     5
8     39377    593     4
9     39417    546     3
计划总数量： 40
账号加权结果： 
    ad_account_id  roi
0           9218    1
1           9219    1
2           9220    1
3           9221    1
4           9224    1
5           9225    1
6           9226    1
7           922

# 测试逻辑（IOS）

## 获取大盘基准

In [134]:
def base_market(amount_thresh):
    """[逻辑：大盘决定 基准配比值 & 素材量]

    Returns:
        [float]: [基准配比值，素材量]
    """
    for day in range(3, 31):
        sum_amount, sum_roi, sum_cost = get_roi(0, day, day) 
        sum_roi = sum_roi * 100
        if sum_amount > amount_thresh*day or day >= 30:  ## 大盘达标 or 近30日大盘
            break
    base_prop, base_num = 5, 30
    print(sum_roi, sum_cost)
    # 规则1：大盘 决定 配比基准值 & 素材基准量
    if sum_cost != 0:
        if (sum_roi <= 0.5) & (sum_cost >= 600):
            base_prop, base_num =2, 5 
        elif ((sum_roi <= 0.5) & (sum_cost < 600)) or ((sum_roi <=0.8) & (sum_cost >= 400)):
            base_prop, base_num =3, 10
        elif ((sum_roi <= 0.8) & (sum_cost < 400)) or ((sum_roi <=1.0) & (sum_cost >= 300)):
            base_prop, base_num =4, 15
        elif ((sum_roi <= 1.0) & (sum_cost < 300)) or ((sum_roi <= 1.5) & (sum_cost >= 200)):
            base_prop, base_num =5, 20
        elif ((sum_roi <= 1.5) & (sum_cost < 200)) or ((sum_roi <= 2.0) & (sum_cost >= 100)):
            base_prop, base_num =6, 25
        else:
            base_prop, base_num =7, 30

    # 规则2：个人盘 修正 极端roi的情况
    for day in range(3, 31):
        sum_amount, sum_roi, sum_cost = get_roi(0, day, day) 
        sum_roi = sum_roi * 100
        if sum_amount > amount_thresh*day or day >= 30:
            if sum_roi > 5.0 or sum_roi < 0.3:
                base_prop, base_num = 3, 10   ## 个人数据异常高、低，不稳定时，采取保守策略
            break
    return base_prop, base_num

In [135]:
base_market(50000)

0.41190000000000004 212.495865


(3, 10)

## 获取总ROI

In [133]:
def get_roi(x, y, z):
    """ 数据：大盘roi与大盘pay_cost两个指标 """

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
       SELECT
            aa.launch_op_id AS 'launch_op_id',
            aa.media_id AS 'media_id',
            sum( aa.amount ) AS 'amount',
            IFNULL( sum( aa.create_role_num ), 0 ) AS 'create_role_num',
            IFNULL( sum( bb.pay_role_user_num ), 0 ) AS 'pay_num',
            IFNULL( sum( bb.new_role_money ), 0 ) AS 'pay_sum',
            (
         CASE
                    WHEN ifnull( sum( aa.pay_role_user_num ), 0 )= 0 THEN
                            IFNULL( sum( aa.amount ), 0 ) ELSE IFNULL( sum( aa.amount ), 0 ) / ifnull( sum( aa.create_role_num ), 0 )
                                                    END 
                                                    ) AS 'create_role_cost', 
            (
         CASE
                    WHEN ifnull( sum( aa.amount ), 0 )= 0 THEN
                            0 ELSE IFNULL( sum( aa.new_role_money ), 0 ) / ifnull( sum( aa.amount ), 0 )
                                                    END 
                                                    ) AS 'create_role_roi' 
        FROM
            (
            SELECT
                a.game_id,
                a.channel_id,
                a.source_id,
                b.launch_op_id,
                a.media_id,
                IFNULL( sum( a.amount ), 0 ) AS amount,
                IFNULL( sum( create_role_num ), 0 ) AS create_role_num,
                IFNULL( sum( a.new_role_money ), 0 ) AS new_role_money,
                IFNULL( sum( a.pay_role_user_num ), 0 ) AS pay_role_user_num

            FROM
                db_stdata.st_lauch_report a
                INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                AND a.source_id = b.source_id 
                AND a.channel_id = b.chl_user_id
            WHERE
                a.tdate_type = 'day' 
                AND a.tdate <= DATE_SUB( date( NOW()), INTERVAL {} DAY )  AND a.tdate >= DATE_SUB( date( NOW()), INTERVAL {} DAY ) 
                AND a.media_id = 16
                AND a.game_id IN ( 1001444,1001446,1001447 ) 
            GROUP BY
                a.game_id,
                a.channel_id,
                a.source_id
            ) aa
            LEFT JOIN (
                SELECT
                    c.game_id,
                    c.channel_id,
                    c.source_id,
                    sum( c.create_role_money ) new_role_money,
                    IFNULL( sum( c.pay_role_user_num ), 0 ) AS pay_role_user_num 
                FROM
                    db_stdata.st_game_days c 
                WHERE
                    c.report_days = {} 
                    AND c.tdate = date( NOW() - INTERVAL 24 HOUR ) 
                    AND c.tdate_type = 'day' 
                    AND c.query_type = 13 
                GROUP BY
                    c.game_id,
                    c.channel_id,
                    c.source_id 
                HAVING
                ( new_role_money > 0 OR pay_role_user_num > 0 )
            ) bb ON aa.game_id = bb.game_id 
            AND aa.channel_id = bb.channel_id 
            AND aa.source_id = bb.source_id 
        GROUP BY
            aa.launch_op_id,
            aa.media_id
    '''
    finalSql = sql.format(x, y, z)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()

    return result_df['amount'].values[0], result_df['create_role_roi'].values[0], result_df['create_role_cost'].values[0]

In [124]:
get_roi(0, 3, 3)  ## 输出：平均每天消耗、总回款、总付费成本

(255003.3, 0.004125, 212.857513)